In [7]:
#from importlib import reload
from bs4 import BeautifulSoup
import requests as requests
import time
#import re
import pandas as pd
import numpy as np
from scripts import user, item, discussion
#import csv
from tqdm import tqdm 

## Test chunck

In [ ]:
titles = []

start = time.time()

for i in range(1,5):
    
    try:
        url_posts = 'https://stacker.news/items/%d' % (i)
        response = requests.get(url_posts)
        soup = BeautifulSoup(response.text, 'html.parser')
    
        title = soup.find_all('a', class_='item_title__FH7AS text-reset me-2')
        
        for e in title:
            titles.append(e.get_text())
        
    except:
        continue
    
end = time.time()

print("The time of execution of above program is :",
      (end-start), "s")
print(titles)

### Collect post header with all the related data

In [ ]:
url_posts = 'https://stacker.news/items/279242'
response = requests.get(url_posts)
soup = BeautifulSoup(response.text, 'html.parser')

titolo = soup.find('a', class_='item_title__FH7AS text-reset me-2', href='/items/279242')


banner = soup.find('div', class_='item_other__MjgP3')

#post_stack = banner.find('span').text

titolo_data = []

# for i in titolo.find('span'):
#     titolo_data.append(i.text)
    
#n_comments = soup.find('a', class_='text-reset position-relative').get_text()
#nym_creator = banner.find('a', ) 

print(titolo==None)


## Scraping Items of stacker.news

### Scraping comments

[Useful link to duplicate rows according to the values in a row. In our situation, we need to duplicate rows according to the usernames or according to the comment item number](https://saturncloud.io/blog/splitting-and-expanding-pandas-dataframes-based-on-column-values/)

In [11]:
url_posts = 'https://stacker.news/items/278856'
response = requests.get(url_posts)
soup = BeautifulSoup(response.text, 'html.parser')

commenti_prova = []

for i in soup.find_all('div', class_='item_item__Q_HbW comment_item__kLv_x'):
    commenti_prova.append(i.get_text())
    
print(commenti_prova[0])

110 sats \ 4 replies \ @siggy47  18hFascinating article. I guess we all can relate to having experienced audience capture. I'm sure I don't even realize how it's affected me and to what extent, but I have noticed it on occasion. It's usually when I write a thoughtless, automatic post or reply, relying on groupthink rather than my own beliefs.
Thanks for posting.


In [6]:
# This code collects the item number of every comment in a post

import re
url_posts = 'https://stacker.news/items/278856'
response = requests.get(url_posts)
soup = BeautifulSoup(response.text, 'html.parser')

commenters = []

for i in tqdm(soup.find_all('a', class_='text-reset position-relative')):
     c = i.get('href')
     r = r'(\D+)'
     res = int(re.sub(r, '', c))
     
     commenters.append(res)
      
print(commenters)

100%|██████████| 7/7 [00:00<00:00, 47051.49it/s]

[278856, 278922, 278948, 279342, 279367, 278961, 279563]


In [11]:
url_posts = 'https://stacker.news/items/278856'
response = requests.get(url_posts)
soup = BeautifulSoup(response.text, 'html.parser')

#discussion.extract_title(soup)
discussion.extract_banner(soup)

['553 sats', ' \\ ', ' \\ ', '@elvismercury  9 Oct', ' ', '', ' ', 'meta', '']

## Testing the extraction of item type

**Since job offers do not store a lot of data about interactions with users (very few comments, only a bunch of job offers into 300k items, etc) we could drop those and do not scrape them**
In fact you can see them by simply looking at the home page of stacker.news and filter for 'job'

In [10]:
for n in [277155, 127070, 277394, 278874, 277840, 239180, 235708, 260050]:
    url_posts = f'https://stacker.news/items/{n}'
    response = requests.get(url_posts)
    soup = BeautifulSoup(response.text, 'html.parser')
    print(n, item.detect_item_type(n, soup))

277155 discussion
127070 bounty
277394 comment
278874 link
277840 discussion
239180 discussion
235708 poll
260050 job


In [ ]:
# # Function to recursively scrape comments and capture hierarchy
# def scrape_comments(comment_element, depth=0):
#     comment_text = comment_element.find('div', class_='comment-text').get_text().strip()
#     
#     # Print the comment with proper indentation based on depth
#     print('  ' * depth + comment_text)
#     
#     # Recursively scrape child comments
#     child_comments = comment_element.find_all('div', class_='comment')
#     for child_comment in child_comments:
#         scrape_comments(child_comment, depth + 1)
# 
# def scrape_post_comments(post_url):
#     response = requests.get(post_url)
#     if response.status_code != 200:
#         print("Failed to fetch the page")
#         return
# 
#     soup = BeautifulSoup(response.text, 'html.parser')
#     
#     # Find and scrape the post content
#     post_content = soup.find('div', class_='post-content').get_text().strip()
#     print("Post Content:")
#     print(post_content)
#     
#     # Find and scrape top-level comments
#     top_level_comments = soup.find_all('div', class_='comment-root')
#     
#     print("\nComments:")
#     for comment in top_level_comments:
#         scrape_comments(comment)
# 
# if __name__ == "__main__":
#     post_url = "https://example.com/post-url"  # Replace with the URL of the post you want to scrape
#     scrape_post_comments(post_url)


## Reframing the previous code
Main steps in the code
1. Retrieve item webpage provided the item code
2. Detect item type:
    - Comment or post ?
    - If post, which kind of post:
        1. Discussion
        2. Link
        3. Poll
        4. Bounty
        5. Job
3. Retrieve title
4. Retrieve banner
    - Extract number of comment, **compulsory**
    - Extract stacked amount by the item, **if present**
    - Extract Boost value, **if present**
    - Extract username, **compulsory**
    - Extract timestamp, **compulsory**
    - Extract badge, **compulsory**
5. Extract amount stacked by comments, **compulsory**
6. Extract item code of comments **OR** extract user that commented

**Note that**:
- Some items do not have the stacked amount nor the possibility to receive sats. For example the user @saloon created all this kind of posts. Is he/she a bot? Is it an 'official bot' of the forum and so it's not possible to give sats to it?


# Scraping user profiles
Users profiles are scraped starting from the list of users extracted by scraping all the items (posts+comments)
The link to get the user profile is `https://stacker.news/$username$` 

In [ ]:
user_list = ['Monotone',
             'TNStacker',
             'kale',
             'DiracDelta',
             'kr',
             'moscowTimeBot',
             'mpuels',
             'blockstream_official',
             'nym',
             0,
             'random_',
             'saloon',
             "k00b",
             "utente che non esiste per niente",
             "DarthCoin",
             "Wumbo",
             "mf",
             "NoStranger",
             "anipy",
             "OneOneSeven",
             "Bitman",
             "nemo",
             "sahil",
             "prova_di_nullo",
             "babababa nullo",
             None,
             ]

user_list2 = ["k00b", "DarthCoin", "saloon"]

start = time.time()

user.save_profile_csv(user_list)

end = time.time()
print("The provided entries are ", len(user_list),"\nThe average time of execution of above program for every entry is :",
      (end-start)/len(user_list), "\nThe total time of execution is ", (end-start))

## Modularize the code - user profile scraping
Define functions to modularize and simplify the user profile scraping.
The functions are defined in `user_modules/scraping_user.py` and have been tested with `tests/test_scraping_user.py`. To run the tests open a the terminal/CMD and run the script `tests/test_scraping_user.py` from there, running it from a JupyterNotebook could raise errors.
The testing script tested every function in some corner cases (missing data/request error). 

**From the following two approaches we can create the final script for scraping the user profiles**.
The next step would be the creation of a function that loops through the user list and assigns to the rows in the dataframe the values returned from the functions defined in `user_modules/scraping_user.py`.

In [ ]:
# TODO: integrate this method into the general scraping script for profile scraping

user_list = np.array(['Monotone',
                      'TNStacker',
                      'kale',
                      'DiracDelta',
                      'kr',
                      'moscowTimeBot',
                      'mpuels',
                      'blockstream_official',
                      'nym',
                      'random_'
                      ])
                      
start = time.time()

for i in np.nditer(user_list):
    print(user.get_profile(i))
    
end = time.time()
print("The average time of execution of the above program for one user is :",
      (end-start)/len(user_list), "s")

In [ ]:
user_list = ['Monotone',
             'TNStacker',
             'kale',
             'DiracDelta',
             'kr',
             'moscowTimeBot',
             'mpuels',
             'blockstream_official',
             'nym',
             'random_',
             ]
                      
start = time.time()

for i in user_list:
    print(user.get_profile(i))
    
end = time.time()
print("The average time of execution of the above program for one user is :",
      (end-start)/len(user_list), "s")